<a href="https://colab.research.google.com/github/deviwlndr/analisis_sentimen/blob/main/analisis_sentimen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Sastrawi
!pip install googletrans==4.0.0-rc1 langdetect



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 28.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.2 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=63a43c6f4313e780c7d60f6792ba116779a0035fbc1cd7ee87a649c907a10c46
  Stored in directory: /root/.cache/pip/wheels

In [2]:
import pandas as pd
import numpy as np
import re
import string
import nltk

from langdetect import detect
from googletrans import Translator

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from wordcloud import WordCloud
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Analisis Sentimen/dataset_ulasan_museum_sri_baduga.csv')
df.head()


,title,url,stars,name,reviewUrl,text
0,Museum Sri Baduga,https://www.google.com/maps/search/?api=1&quer...,4,NaN,NaN,NaN
1,Museum Sri Baduga,https://www.google.com/maps/search/?api=1&quer...,5,NaN,NaN,Musieum Sribaduga
2,Museum Sri Baduga,https://www.google.com/maps/search/?api=1&quer...,5,NaN,NaN,NaN
3,Museum Sri Baduga,https://www.google.com/maps/search/?api=1&quer...,5,NaN,NaN,Museum di dalam kota bandung
4,Museum Sri Baduga,https://www.google.com/maps/search/?api=1&quer...,4,NaN,NaN,هذا المتحف هو رحلة عبر الزمن، حيث نرى بعض من ت...


In [5]:
df = df[['text', 'stars']]
df = df.dropna(subset=['text'])
df['stars'] = df['stars'].astype(int)

df.head()


,text,stars
1,Musieum Sribaduga,5
3,Museum di dalam kota bandung,5
4,هذا المتحف هو رحلة عبر الزمن، حيث نرى بعض من ت...,4
7,"Kesini pas study tour SMP, mungkin tahun 2018/...",5
8,Bagus sekali,5


In [6]:
def case_folding(text):
    return text.lower()

In [7]:
df['case_folding'] = df['text'].apply(case_folding)

In [73]:
df[["text", "case_folding"]].head(20)

,text,case_folding
1,Musieum Sribaduga,musieum sribaduga
3,Museum di dalam kota bandung,museum di dalam kota bandung
4,هذا المتحف هو رحلة عبر الزمن، حيث نرى بعض من ت...,هذا المتحف هو رحلة عبر الزمن، حيث نرى بعض من ت...
7,"Kesini pas study tour SMP, mungkin tahun 2018/...","kesini pas study tour smp, mungkin tahun 2018/..."
8,Bagus sekali,bagus sekali
9,Mantap,mantap
10,Seru anak jadi tau sejarah dan banyak belajar,seru anak jadi tau sejarah dan banyak belajar
16,mantap,mantap
17,Tempat ini sangat baik untuk edukasi anak didi...,tempat ini sangat baik untuk edukasi anak didi...
18,Datang ke sini sewaktu ada Pameran Nasional Pu...,datang ke sini sewaktu ada pameran nasional pu...


In [8]:
# 3. CHARACTER FILTERING
# Menghapus angka, emoji, tanda baca, URL
# ---------------------------------------------
def clean_characters(text):
    text = re.sub(r'http\S+|www\S+|https\S+', ' ', text)   # hapus URL
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)              # hapus angka, emoji, simbol
    text = re.sub(r'\s+', ' ', text).strip()              # rapikan spasi
    return text

In [9]:
df['clean_text'] = df['case_folding'].apply(clean_characters)

In [75]:
df[['case_folding', 'clean_text']].head(20)

,case_folding,clean_text
1,musieum sribaduga,musieum sribaduga
3,museum di dalam kota bandung,museum di dalam kota bandung
4,هذا المتحف هو رحلة عبر الزمن، حيث نرى بعض من ت...,
7,"kesini pas study tour smp, mungkin tahun 2018/...",kesini pas study tour smp mungkin tahun
8,bagus sekali,bagus sekali
9,mantap,mantap
10,seru anak jadi tau sejarah dan banyak belajar,seru anak jadi tau sejarah dan banyak belajar
16,mantap,mantap
17,tempat ini sangat baik untuk edukasi anak didi...,tempat ini sangat baik untuk edukasi anak didi...
18,datang ke sini sewaktu ada pameran nasional pu...,datang ke sini sewaktu ada pameran nasional pu...


In [10]:
translator = Translator()

def translate_to_indonesian(text):
    if not isinstance(text, str) or text.strip() == "":
        return text

    try:
        lang = detect(text)
    except:
        return text

    if lang == "en":  # hanya terjemahkan jika English
        try:
            return translator.translate(text, src='en', dest='id').text
        except:
            return text

    return text



In [11]:
df['translated'] = df['clean_text'].apply(translate_to_indonesian)
print(df[['clean_text','translated']].head())


                                clean_text  \
1                        musieum sribaduga   
3             museum di dalam kota bandung   
4                                            
7  kesini pas study tour smp mungkin tahun   
8                             bagus sekali   

                                translated  
1                        musieum sribaduga  
3             museum di dalam kota bandung  
4                                           
7  kesini pas study tour smp mungkin tahun  
8                             bagus sekali  


In [77]:
df[['clean_text', 'translated']].head(50)

,clean_text,translated
1,musieum sribaduga,musieum sribaduga
3,museum di dalam kota bandung,museum di dalam kota bandung
4,,
7,kesini pas study tour smp mungkin tahun,kesini pas study tour smp mungkin tahun
8,bagus sekali,bagus sekali
9,mantap,mantap
10,seru anak jadi tau sejarah dan banyak belajar,seru anak jadi tau sejarah dan banyak belajar
16,mantap,mantap
17,tempat ini sangat baik untuk edukasi anak didi...,tempat ini sangat baik untuk edukasi anak didi...
18,datang ke sini sewaktu ada pameran nasional pu...,datang ke sini sewaktu ada pameran nasional pu...


In [12]:
# 4. TOKENIZING

nltk.download('punkt')

def tokenizing(text):
    return nltk.word_tokenize(text)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
df['tokenized'] = df['translated'].apply(tokenizing)

In [79]:
df[['translated', 'tokenized']].head(20)

,translated,tokenized
1,musieum sribaduga,"[musieum, sribaduga]"
3,museum di dalam kota bandung,"[museum, di, dalam, kota, bandung]"
4,,[]
7,kesini pas study tour smp mungkin tahun,"[kesini, pas, study, tour, smp, mungkin, tahun]"
8,bagus sekali,"[bagus, sekali]"
9,mantap,[mantap]
10,seru anak jadi tau sejarah dan banyak belajar,"[seru, anak, jadi, tau, sejarah, dan, banyak, ..."
16,mantap,[mantap]
17,tempat ini sangat baik untuk edukasi anak didi...,"[tempat, ini, sangat, baik, untuk, edukasi, an..."
18,datang ke sini sewaktu ada pameran nasional pu...,"[datang, ke, sini, sewaktu, ada, pameran, nasi..."


In [14]:
# Kamus normalisasi (bisa ditambah)
normalization_dict = {
    # --- Kata Informal / Slang ---
    "ga": "tidak",
    "gk": "tidak",
    "gak": "tidak",
    "tdk": "tidak",
    "engga": "tidak",
    "enggak": "tidak",
    "nggak": "tidak",
    "ngga": "tidak",

    "bgt": "banget",
    "bngt": "banget",
    "bangettt": "banget",

    "cmn": "cuma",
    "cman": "cuma",
    "cuma2": "cuma",

    "trs": "terus",
    "trus": "terus",

    "tmn": "teman",
    "tmn2": "teman",

    "udh": "sudah",
    "udah": "sudah",
    "udhh": "sudah",

    "blm": "belum",
    "belom": "belum",

    "syg": "sayang",

    "krg": "kurang",

    "sblm": "sebelum",

    "dr": "dari",
    "drpd": "daripada",

    "klo": "kalau",
    "kalo": "kalau",

    "pls": "tolong",
    "plis": "tolong",

    "aja": "saja",
    "aj": "saja",

    "ni": "ini",
    "nih": "ini",

    "tu": "itu",
    "ituu": "itu",

    "bru": "baru",
    "baru2": "baru",

    "lok": "lokasi",

    "pas": "saat",

    # --- Singkatan Lokasi / Umum ---
    "bndng": "bandung",
    "bdg": "bandung",
    "jbr": "jawa barat",

    # --- Typos Umum ---
    "bgs": "bagus",
    "baguss": "bagus",
    "bgus": "bagus",

    "bnyk": "banyak",
    "bnyknya": "banyaknya",

    "tmpt": "tempat",
    "tempet": "tempat",

    "rame": "ramai",
    "ramee": "ramai",

    "nyamn": "nyaman",
    "nyamann": "nyaman",

    "lumayanlah": "lumayan",

    "mantappp": "mantap",
    "mantab": "mantap",
    "mantul": "mantap",

    # --- Istilah Museum / Edukasi ---
    "edukasii": "edukasi",
    "edukatif": "edukatif",
    "koleksinyaa": "koleksinya",
    "sejarahh": "sejarah",

    # --- Bahasa Inggris (umum di Google Maps) ---
    "good": "bagus",
    "nice": "bagus",
    "great": "bagus",
    "awesome": "bagus",
    "amazing": "bagus",
    "cool": "keren",
    "clean": "bersih",
    "recommended": "direkomendasikan",
    "recommend": "rekomendasi",
    "worth": "layak",
    "worthit": "layak",
    "worthwhile": "layak",
    "cheap": "murah",
    "expensive": "mahal",
    "great": "bagus",
    "nice": "bagus",

    # --- Bahasa Sunda (umum dalam ulasan Museum Sri Baduga) ---
    "teh": "",         # partikel (dapat dihapus)
    "mah": "",         # partikel penegas
    "atuh": "",        # ekspresi
    "pisan": "sekali",
    "teteh": "kakak",
    "kenging": "boleh",
    "alus": "bagus",
    "gawe": "kerja",

    # --- Variasi ejaan tidak baku ---
    "seblm": "sebelum",
    "seblumnya": "sebelumnya",
    "baguss": "bagus",
    "bersiih": "bersih",
    "bersiiih": "bersih",
    "bersihh": "bersih",

    "murahh": "murah",
    "murahhh": "murah",

    "lengkappp": "lengkap",

    "worthittt": "layak",

    # --- Kata sambung / filler ---
    "the": "the",          # tetap (kadang judul)
    "of": "of",            # tetap
    "ya": "",              # filler
    "yah": "",             # filler
    "loh": "",             # filler
    "kok": "",             # filler
    "kan": "",             # filler
    "lah": "",             # filler

    # --- Variasi Pengulangan Huruf (contoh umum) ---
    "bagusss": "bagus",
    "kerenn": "keren",
    "keren banget": "keren",
    "kenyanggg": "kenyang",
    "luasss": "luas",
    "seruu": "seru",
    "seruuu": "seru",
    "harganyaa": "harganya",

    # --- Kata Google Maps yang sering muncul ---
    "service": "layanan",
    "staff": "staf",
    "facility": "fasilitas",
    "facilities": "fasilitas",
    "parking": "parkir",
    "parkiran": "parkir",
    "recommended": "direkomendasikan",
    "experience": "pengalaman",
    "view": "pemandangan",
    "history": "sejarah",
    "historical": "sejarah",

    # --- Tambahan umum ---
    "yg": "yang",
    "aja": "saja",
    "gini": "begini",
    "gitu": "begitu",
    "sini": "sini",
    "situ": "situ",
    "kemaren": "kemarin",
    "kmrn": "kemarin",
    "kmrn2": "kemarin",
    "besok2": "besok",
    "jd": "jadi",
    "jg": "juga",
    "tll": "terlalu",
    "sgt": "sangat"
}

def normalize_text(tokens):
    return [normalization_dict.get(word, word) for word in tokens]


In [15]:
df['normalized'] = df['tokenized'].apply(normalize_text)

In [81]:
df[['tokenized', 'normalized']].head(50)

,tokenized,normalized
1,"[musieum, sribaduga]","[musieum, sribaduga]"
3,"[museum, di, dalam, kota, bandung]","[museum, di, dalam, kota, bandung]"
4,[],[]
7,"[kesini, pas, study, tour, smp, mungkin, tahun]","[kesini, saat, study, tour, smp, mungkin, tahun]"
8,"[bagus, sekali]","[bagus, sekali]"
9,[mantap],[mantap]
10,"[seru, anak, jadi, tau, sejarah, dan, banyak, ...","[seru, anak, jadi, tau, sejarah, dan, banyak, ..."
16,[mantap],[mantap]
17,"[tempat, ini, sangat, baik, untuk, edukasi, an...","[tempat, ini, sangat, baik, untuk, edukasi, an..."
18,"[datang, ke, sini, sewaktu, ada, pameran, nasi...","[datang, ke, sini, sewaktu, ada, pameran, nasi..."


In [16]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factory = StopWordRemoverFactory()
stopwords_id = set(factory.get_stop_words())

additional_stopwords = {
    "banget", "sekali", "kali",
    "ya", "yah", "iya",
    "sih", "nih", "tuh",
    "kayak", "kayaknya",
    "dong", "deh", "kok",
    "gitu", "gini",
    "aja", "doang",
    "bikin", "biar", "buat", "jadi",
    "tempat", "lokasi", "area",
    "museum", "to", "we", "about"
}

stopwords_final = stopwords_id.union(additional_stopwords)

def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopwords_final]



In [17]:
df['stopword_removed'] = df['normalized'].apply(remove_stopwords)

In [82]:
df[['normalized', 'stopword_removed']].head(50)

,normalized,stopword_removed
1,"[musieum, sribaduga]","[musieum, sribaduga]"
3,"[museum, di, dalam, kota, bandung]","[kota, bandung]"
4,[],[]
7,"[kesini, saat, study, tour, smp, mungkin, tahun]","[kesini, study, tour, smp, mungkin, tahun]"
8,"[bagus, sekali]",[bagus]
9,[mantap],[mantap]
10,"[seru, anak, jadi, tau, sejarah, dan, banyak, ...","[seru, anak, tau, sejarah, banyak, belajar]"
16,[mantap],[mantap]
17,"[tempat, ini, sangat, baik, untuk, edukasi, an...","[sangat, baik, edukasi, anak, didik, koleksiny..."
18,"[datang, ke, sini, sewaktu, ada, pameran, nasi...","[datang, sini, sewaktu, pameran, nasional, pus..."


In [18]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_sentence(tokens):
    sentence = " ".join(tokens)         # gabungkan list → string
    stemmed = stemmer.stem(sentence)    # stemming kalimat
    return stemmed.split()              # pecah lagi → list




In [19]:
df["stemmed_text"] = df["stopword_removed"].apply(stem_sentence)
df['stemmed_joined'] = df['stemmed_text'].apply(lambda x: " ".join(x))

In [83]:
df[['stopword_removed', 'stemmed_text', 'stemmed_joined']].head(50)

,stopword_removed,stemmed_text,stemmed_joined
1,"[musieum, sribaduga]","[musieum, sribaduga]",musieum sribaduga
3,"[kota, bandung]","[kota, bandung]",kota bandung
4,[],[],
7,"[kesini, study, tour, smp, mungkin, tahun]","[kesini, study, tour, smp, mungkin, tahun]",kesini study tour smp mungkin tahun
8,[bagus],[bagus],bagus
9,[mantap],[mantap],mantap
10,"[seru, anak, tau, sejarah, banyak, belajar]","[seru, anak, tau, sejarah, banyak, ajar]",seru anak tau sejarah banyak ajar
16,[mantap],[mantap],mantap
17,"[sangat, baik, edukasi, anak, didik, koleksiny...","[sangat, baik, edukasi, anak, didik, koleksi, ...",sangat baik edukasi anak didik koleksi lengkap
18,"[datang, sini, sewaktu, pameran, nasional, pus...","[datang, sini, waktu, pamer, nasional, pusaka,...",datang sini waktu pamer nasional pusaka nusant...


In [20]:
import pandas as pd

# URL resmi InSet Lexicon
url_positive = "https://raw.githubusercontent.com/fajri91/InSet/master/positive.tsv"
url_negative = "https://raw.githubusercontent.com/fajri91/InSet/master/negative.tsv"

# Membaca lexicon dari GitHub tanpa header bawaan file
lex_pos = pd.read_csv(url_positive, sep="\t", header=None, names=["word", "weight"], skiprows=1)
lex_neg = pd.read_csv(url_negative, sep="\t", header=None, names=["word", "weight"], skiprows=1)

# Gabungkan
lexicon_df = pd.concat([lex_pos, lex_neg], ignore_index=True)

# Konversi weight ke integer (AMAN sekarang)
lexicon_df["weight"] = lexicon_df["weight"].astype(int)

# Jadikan dictionary
lexicon = dict(zip(lexicon_df["word"], lexicon_df["weight"]))

print("Jumlah kata dalam lexicon:", len(lexicon))


Jumlah kata dalam lexicon: 9074


In [21]:
# Ambil bobot maksimum untuk tiap kata → lebih logis untuk sentimen
lexicon_clean = lexicon_df.groupby("word", as_index=False)["weight"].max()

# Ubah ke dictionary
lexicon_final = dict(zip(lexicon_clean["word"], lexicon_clean["weight"]))

print("Jumlah kata di lexicon_final:", len(lexicon_final))


Jumlah kata di lexicon_final: 9074


In [22]:
def get_lexicon_score(tokens, lexicon):
    if not isinstance(tokens, list):
        return 0
    return sum(lexicon.get(word, 0) for word in tokens)

df["lexicon_score"] = df["stemmed_text"].apply(lambda x: get_lexicon_score(x, lexicon_final))


In [23]:
def label_sentiment(score):
    if score > 0:
        return "positif"
    elif score < 0:
        return "negatif"
    else:
        return "netral"

df["sentiment"] = df["lexicon_score"].apply(label_sentiment)


In [24]:
df[["stemmed_text", "lexicon_score", "sentiment"]].head(20)


,stemmed_text,lexicon_score,sentiment
1,"[musieum, sribaduga]",0,netral
3,"[kota, bandung]",-2,negatif
4,[],0,netral
7,"[kesini, study, tour, smp, mungkin, tahun]",-1,negatif
8,[bagus],2,positif
9,[mantap],5,positif
10,"[seru, anak, tau, sejarah, banyak, ajar]",2,positif
16,[mantap],5,positif
17,"[sangat, baik, edukasi, anak, didik, koleksi, ...",9,positif
18,"[datang, sini, waktu, pamer, nasional, pusaka,...",3,positif


In [25]:
df['sentiment'].value_counts().reset_index().rename(columns={'index': 'sentiment', 'sentiment': 'jumlah'})


,jumlah,count
0,positif,1045
1,negatif,228
2,netral,219


In [59]:
from sklearn.model_selection import train_test_split

X = df['stemmed_joined']    # fitur berupa teks
y = df['sentiment']         # label hasil lexicon

X_train_text, X_test_text, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Jumlah data latih:", len(X_train_text))
print("Jumlah data uji:", len(X_test_text))
print("Distribusi train:\n", y_train.value_counts())
print("Distribusi test:\n", y_test.value_counts())


Jumlah data latih: 1193
Jumlah data uji: 299
Distribusi train:
 sentiment
positif    836
negatif    182
netral     175
Name: count, dtype: int64
Distribusi test:
 sentiment
positif    209
negatif     46
netral      44
Name: count, dtype: int64


In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    min_df=2
)

# Fit hanya pada data latih
X_train_tfidf = tfidf.fit_transform(X_train_text)

# Transform pada data uji
X_test_tfidf = tfidf.transform(X_test_text)

print("Shape train TF-IDF:", X_train_tfidf.shape)
print("Shape test TF-IDF:", X_test_tfidf.shape)


Shape train TF-IDF: (1193, 1805)
Shape test TF-IDF: (299, 1805)


In [67]:
# Jumlah kolom (fitur) = 1805 pada kedua dataset
# Artinya setiap dokumen sudah berhasil dikonversi menjadi vektor berdimensi sama, yaitu 1.805 fitur.
# Ini menunjukkan tidak ada error, tidak ada dokumen yang gagal diproses, dan tidak terjadi leakage fitur (karena TF-IDF di-fit pada keseluruhan data sebelum split).
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)


(1193, 1805)
(299, 1805)


In [62]:
#tidak ada misisng value
df.isnull().values.any()


np.False_

In [69]:
features = tfidf.get_feature_names_out()
print(features[:50])  # contoh 50 fitur pertama
print(len(features))  # jumlah fitur



['abad' 'abadi' 'abadi buah' 'ac' 'acara' 'ad' 'ada' 'ada jawa'
 'ada tengah' 'adab' 'adab jawa' 'adab manusia' 'adat' 'adem' 'aga'
 'agama' 'air' 'air kecil' 'ajak' 'ajak anak' 'ajak main' 'ajar'
 'ajar anak' 'ajar asal' 'ajar bagai' 'ajar bandung' 'ajar banyak'
 'ajar budaya' 'ajar ilmu' 'ajar jaman' 'ajar lebih' 'ajar mahasiswa'
 'ajar sangat' 'ajar sayang' 'ajar sejarah' 'ajar tiket' 'ajar wajib'
 'akhir' 'akhir kesini' 'akhir tahun' 'aksara' 'akses' 'akses mudah'
 'akses nya' 'aku' 'al' 'alam' 'alam ajar' 'alam budaya' 'alam sejarah']
1805


In [70]:
print(tfidf.vocabulary_)   # fitur + index


{'mantap': np.int64(983), 'lumayan': np.int64(952), 'bagus': np.int64(153), 'lumayan bagus': np.int64(953), 'tempat': np.int64(1636), 'utk': np.int64(1750), 'rekreasi': np.int64(1293), 'sekaligus': np.int64(1454), 'ajar': np.int64(21), 'bagus tempat': np.int64(183), 'sekaligus ajar': np.int64(1455), 'cukup': np.int64(415), 'bersih': np.int64(324), 'informatif': np.int64(631), 'sejarah': np.int64(1408), 'kembang': np.int64(748), 'jawa': np.int64(687), 'barat': np.int64(252), 'jaman': np.int64(665), 'purbakala': np.int64(1253), 'adab': np.int64(9), 'modern': np.int64(1040), 'tiket': np.int64(1670), 'anak': np.int64(65), 'rp': np.int64(1310), 'dewasa': np.int64(468), 'sejarah kembang': np.int64(1429), 'jawa barat': np.int64(688), 'barat jaman': np.int64(260), 'jaman purbakala': np.int64(671), 'tiket anak': np.int64(1671), 'tiket dewasa': np.int64(1672), 'dewasa rp': np.int64(471), 'musium': np.int64(1080), 'nya': np.int64(1120), 'bagus musium': np.int64(171), 'musium nya': np.int64(1081),

In [71]:
#matriks TF-IDF berhasil terbentuk dan siap digunakan untuk pelatihan model
#output: TF-IDF berhasil membentuk vektor fitur dalam bentuk matriks sparse CSR (Compressed Sparse Row).
print(type(X_train_tfidf))


<class 'scipy.sparse._csr.csr_matrix'>


In [72]:
print(X_test_tfidf.shape)


(299, 1805)


In [26]:
tfidf = TfidfVectorizer(
    ngram_range=(1,2),   # unigram + bigram
    min_df=2,            # kata minimal muncul 2 dokumen
    max_df=0.95          # membuang kata terlalu umum
)


In [27]:
X_tfidf = tfidf.fit_transform(df["stemmed_joined"])


In [28]:
df["sentiment"]   # hasil dari lexicon (positif/negatif/netral)


,sentiment
1,netral
3,negatif
4,netral
7,negatif
8,positif
...,...
3644,positif
3645,positif
3648,positif
3655,negatif


In [29]:
y = df["sentiment"]


In [30]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)


In [31]:
feature_names = tfidf.get_feature_names_out()
feature_names[:50]   # tampilkan 50 kata pertama


array(['abad', 'abadi', 'abadi buah', 'ac', 'acara', 'ad', 'ada',
       'ada depan', 'ada jawa', 'ada tengah', 'adab', 'adab jawa',
       'adab manusia', 'adab masa', 'adat', 'adem', 'aga', 'agam',
       'agama', 'air', 'air kecil', 'ajak', 'ajak anak', 'ajak main',
       'ajar', 'ajar ambil', 'ajar anak', 'ajar asal', 'ajar bagai',
       'ajar bandung', 'ajar banyak', 'ajar budaya', 'ajar ilmu',
       'ajar jaman', 'ajar kenal', 'ajar kisah', 'ajar lebih',
       'ajar mahasiswa', 'ajar main', 'ajar sangat', 'ajar sayang',
       'ajar sejarah', 'ajar tiket', 'ajar ttg', 'ajar wajib', 'akhir',
       'akhir kesini', 'akhir tahun', 'aksara', 'akses'], dtype=object)

In [32]:
doc_index = 0  # pilih dokumen ke-1
df_tfidf_doc = pd.DataFrame(
    X_tfidf[doc_index].T.toarray(),
    index=feature_names,
    columns=["TF-IDF"]
)

# Tampilkan kata dengan bobot terbesar
df_tfidf_doc.sort_values("TF-IDF", ascending=False).head(20)


,TF-IDF
musieum sribaduga,0.641758
musieum,0.596289
sribaduga,0.482273
pasar,0.000000
pasundan,0.000000
patung,0.000000
patung rem,0.000000
pawon,0.000000
panas,0.000000
pegawai,0.000000


In [33]:
#Cek beberapa fitur dengan nilai bobot
#Menampilkan bobot kata pada dokumen tertentu & Memastikan ada nilai TF-IDF yang bervariasi (tidak semuanya 0)
import numpy as np

tfidf_mean = np.mean(X_tfidf.toarray(), axis=0)
top_indices = np.argsort(tfidf_mean)[-20:]

for i in top_indices:
    print(feature_names[i], tfidf_mean[i])


koleksi 0.01431844829873956
nya 0.015227106525286012
sunda 0.015358444998191298
jawa barat 0.016394206441889356
budaya 0.016427744821936038
bersih 0.016528394531947658
barat 0.016665142806147736
jawa 0.016706764543978403
tahu 0.0174708283743791
tempat 0.018153541124180078
bandung 0.018483907811402415
sangat 0.019222637445174516
mantap 0.019693664640020238
edukasi 0.019704081018845573
nyaman 0.019763628696158074
ajar 0.021704470616468637
banyak 0.02269356581309058
anak 0.0251105525748664
sejarah 0.05359647825839011
bagus 0.0842999729310027


In [34]:
df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=feature_names)
df_tfidf.head()


,abad,abadi,abadi buah,ac,acara,ad,ada,ada depan,ada jawa,ada tengah,...,yo,you,yuk,yusuf,zaman,zaman batu,zaman dulu,zaman kala,zaman purba,zaman purbakala
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
#Artinya TF-IDF berhasil menghasilkan 5000 fitur untuk 1492 dokumen.
print("TF-IDF shape:", X_tfidf.shape)


TF-IDF shape: (1492, 2063)


In [36]:
#Matriks TF-IDF bersifat sparse (banyak 0), tapi harus memiliki beberapa nilai bukan nol.
#Jika hasilnya > 0, maka pembobotan berhasil.
X_tfidf.nnz


14713

In [37]:
# Interpretasi density kamu (0.00478)
# Matriks TF-IDF sangat sparse (bagus)
# Preprocessing berhasil (karena ada pola kata tetapi tidak terlalu padat)
# TF-IDF berfungsi normal
# SVM Linear akan bekerja optimal
# Nilai ini persis seperti matriks TF-IDF pada penelitian sentimen real (biasanya density 0.1% – 1%).

density = X_tfidf.nnz / (X_tfidf.shape[0] * X_tfidf.shape[1])
density


0.004780058193707854

In [38]:
#mengecek missing value
X_tfidf = tfidf.fit_transform(df['stemmed_joined'])


In [39]:
df['stemmed_joined'].isnull().sum()


np.int64(0)

In [40]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)


In [41]:
print("Data latih (X_train):", X_train.shape[0])
print("Data uji (X_test):", X_test.shape[0])
print("Label latih (y_train):", len(y_train))
print("Label uji (y_test):", len(y_test))


Data latih (X_train): 1193
Data uji (X_test): 299
Label latih (y_train): 1193
Label uji (y_test): 299


In [46]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE

# ---------------------------
# 1. Pisahkan data terlebih dahulu (anti leakage)
# ---------------------------
X = df['stemmed_joined']      # fitur berupa teks
y = df['sentiment']           # label sentimen

X_train_text, X_test_text, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ---------------------------
# 2. TF-IDF hanya pada data latih
# ---------------------------
tfidf = TfidfVectorizer()

# fit + transform hanya pada data latih
X_train_tfidf = tfidf.fit_transform(X_train_text)

# hanya transform pada data uji
X_test_tfidf = tfidf.transform(X_test_text)

# ---------------------------
# 3. SMOTE hanya pada data latih (anti leakage)
# ---------------------------
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_tfidf, y_train)

In [58]:
print("Sebelum SMOTE:")
print(y_train.value_counts())

print("\nSesudah SMOTE:")
print(pd.Series(y_train_smote).value_counts())


Sebelum SMOTE:
sentiment
positif    836
negatif    182
netral     175
Name: count, dtype: int64

Sesudah SMOTE:
sentiment
positif    836
netral     836
negatif    836
Name: count, dtype: int64


In [57]:
# -------------------------------------------
# 1. AMBIL FITUR & LABEL
# -------------------------------------------

X = df["stemmed_joined"]
y = df["sentiment"]   # gunakan label lexicon terbaru


# -------------------------------------------
# 2. TRAIN TEST SPLIT (lebih aman sebelum TF-IDF)
# -------------------------------------------

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Jumlah data train:", len(X_train))
print("Jumlah data test :", len(X_test))


# -------------------------------------------
# 3. TF-IDF HANYA FIT PADA DATA TRAIN
# -------------------------------------------

tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1,2),
    min_df=2
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf  = tfidf.transform(X_test)   # tidak fit_transform!


# -------------------------------------------
# 4. TRAIN MODEL LinearSVC
# -------------------------------------------

model = LinearSVC()
model.fit(X_train_tfidf, y_train)

# -------------------------------------------
# 5. EVALUASI MODEL
# -------------------------------------------

y_pred = model.predict(X_test_tfidf)

print("Akurasi:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Jumlah data train: 1193
Jumlah data test : 299
Akurasi: 0.8461538461538461
              precision    recall  f1-score   support

     negatif       0.82      0.50      0.62        46
      netral       0.65      0.68      0.67        44
     positif       0.89      0.96      0.92       209

    accuracy                           0.85       299
   macro avg       0.79      0.71      0.74       299
weighted avg       0.84      0.85      0.84       299

